# Ecuaciones de optimalidad de Bellman

<img src="bellman1.png">
<img src="bellman2.png">

# Iteración de función de valor:(del libro de Sutton)

<img src="iteracion de funcion de valor.png">

Vamos a usar la ecuación de optimalidad para encontrar de manera iterativa la política óptima.

In [1]:
# en caso de correrlo en google colab
# de esta manera podremos tener la carpeta lib (donde se encuentra en ambiente Gridworld)

# import sys
#if "../" not in sys.path:
#  sys.path.append("../") 

# !git clone https://github.com/julianfm7/cursoRL-FIUBA

# necesario en google colab para que sys.path busque
# y encuentre la carpeta lib donde se encuentra el ambiente Gridworld

# !mv cursoRL-FIUBA cursoRLFIUBA

import numpy as np
import pprint

from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [10]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Algoritmo de iteración de la función de valor
    
    Argumentos:
        env: ambiente de OpenAI. env.P representa la transición de probabilidades del ambiente.
            env.P[s][a] es una lista de tuplas de transición (prob, next_state, reward, done).
            env.nS es el número de estados del ambiente.
            env.nA es el número de acciones del ambiente.
        theta: Paramos de evaluar una vez que la función de valor cambia menos que theta para todos los estados.
        discount_factor: factor de descuento gama.
        
    Retorna:
        Una tupla (policy, V) de política óptima y función de valor óptima.
    """
    
    def espectationActions_givenState(estado,V):
        lista = np.zeros(env.nA)
        # calcular el valor esperado de cada acción utilizando la función de valor actual y haciendo 'one-step look-ahead'
        for accion in env.P[estado]:
            vact=0
            for probabilidad, proximo_estado, recompensa, _ in env.P[estado][accion]:
                vact += probabilidad*(recompensa + discount_factor*V[proximo_estado])
            lista[accion]=vact
        return lista

    
    Vop = np.zeros(env.nS)
    # primero calcular la funcion valor hasta que el cambio maximo de V sea menor a theta
    while True:
        # por cada estado s
        delta = 0
        for estado in range(env.nS):
            
            # elegir la accion con mayor retorno
            v_op = max(espectationActions_givenState(estado,Vop))

            delta = max(delta,abs(v_op-Vop[estado]))
            Vop[estado] = v_op
        if delta < theta:
            break

          # mantener una variable (delta) que indica el cambio maximo en V(s) para todos los estados
          # hacer update de V(s) = maximo retorno esperado (mejor accion)
        # si delta < theta: parar
    
    
    policy = np.zeros([env.nS, env.nA])
    # ahora calcular la política óptima:
    # por cada estado s
    for estado in range(env.nS):
            
            # encontrar la acción con mayor retorno esperado a_max
            a_op = np.argmax(espectationActions_givenState(estado,Vop))

            # la política debe realizar esta acción a_max en el estado s con probabilidad 1.0
            policy[estado] = [0]*env.nA
            policy[estado,a_op] = 1.0
    
    
    return policy, Vop

In [11]:
policy, v = value_iteration(env)

print("Distribución de probabilidad de la política:")
print(policy)
print("")

print("Política mostrada en la grilla: (0=arriba, 1=derecha, 2=abajo, 3=izquierda)")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Función de valor:")
print(v)
print("")

print("Función de valor exhibida en la grilla:")
print(v.reshape(env.shape))
print("")

Distribución de probabilidad de la política:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Política mostrada en la grilla: (0=arriba, 1=derecha, 2=abajo, 3=izquierda)
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Función de valor:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Función de valor exhibida en la grilla:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [12]:
# Testear la función de valor
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)